In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import datasets

In [10]:
ds = datasets.load_dataset("franlucc/py_caa", split="train")
dsc = datasets.load_dataset("franlucc/py-caa-completions", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/265879 [00:00<?, ? examples/s]

In [5]:
ds2 = ds.map(lambda x: {**x, 
                        "recon2" : x["prefix"] + x["middle"] + x["suffix"],
                        "recon": x["fim_program"].replace("<FILL>", x["fim_type"])})

In [15]:
py_caa_v2 = ds.filter(lambda x: len(tokenizer(x["fim_type"])["input_ids"]) == 1)

Filter:   0%|          | 0/265879 [00:00<?, ? examples/s]

In [18]:
py_caa_v2.push_to_hub("franlucc/py_typeinf_may3-1tok_starcoderbase-1b_completions", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/71 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/71 [00:00<?, ?ba/s]

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/mnt/ssd/arjun/models/starcoderbase-1b")

ds2 = ds2.map(lambda x: {**x, "num_tok" : len(tokenizer(x["fim_type"])["input_ids"])})

Map:   0%|          | 0/265879 [00:00<?, ? examples/s]

In [16]:
ds = datasets.load_from_disk("/home/franlucc/projects_data/codetrace/results_v2/starcoderbase-1b/python/all_muts/ood_steering_results_ds")

In [14]:
dsc["correct"] == ds["correct"]

True

In [20]:
print(ds[2]["fim_program"])

#!/usr/bin/env python3
# Copyright (C) 2021 Checkmk GmbH - License: GNU General Public License v2
# This file is part of Checkmk (https://checkmk.com). It is subject to the terms and
# conditions defined in the file COPYING, which is part of this source code package.

from collections.abc import Sequence

import pytest
from polyfactory.factories.pydantic_factory import ModelFactory
from pytest_mock import MockerFixture

from cmk.base.plugins.agent_based.agent_based_api.v1 import Attributes, TableRow
from cmk.base.plugins.agent_based.inventory_kube_node import inventory_kube_node

from cmk.plugins.lib.kube import (
    HealthZ,
    IpAddress,
    KubeletInfo,
    NodeAddress,
    NodeInfo,
    NodeName,
    Timestamp,
)

from .utils_inventory import sort_inventory_result


@pytest.mark.parametrize(
    "section_info, section_kubelet, expected_check_result",
    [
        pytest.param(
            NodeInfo(
                architecture="amd64",
                kernel_version="5.13.0-27-g